<a href="https://colab.research.google.com/github/mspsullivan/DataCleaningProject/blob/master/fakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

import seaborn as sns

import re
import nltk
nltk.download(['stopwords', 'punkt','wordnet','omw-1.4'])
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
df = pd.read_csv('public_kaggle_fake_news_training_set.csv', encoding = 'unicode_escape')
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,id,title,author,text,label
0,0,House Dem Aide: We Didn??t Even See Comey??s...,Darrell Lucus,House Dem Aide: We Didn??t Even See Comey??s...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.callbacks import EarlyStopping
x_train, x_test, y_train, y_test = train_test_split(df['text'],
  df['label'],
  test_size=0.2,
  random_state=42)

In [5]:
maxlen = 128
truncating = 'post'
padding = 'post'
oov_tok = '<OOV>'
vocab_size = 1000

tokenizer = Tokenizer(num_words = vocab_size,
                      char_level = False,
                      oov_token = oov_tok)
tokenizer.fit_on_texts(x_train)

In [6]:
training_sequences = tokenizer.texts_to_sequences(x_train)
training_sequences = pad_sequences(training_sequences,
                                   maxlen = maxlen,
                                   padding = padding,
                                   truncating = truncating)
testing_sequences = tokenizer.texts_to_sequences(x_test)
testing_sequences = pad_sequences(testing_sequences,
                                  maxlen = maxlen,
                                  padding = padding,
                                  truncating = truncating)

In [7]:
#print(x_train)
print(testing_sequences.shape)
print(training_sequences.shape)

(3, 128)
(11, 128)


In [8]:

from tensorflow import keras
embedding_dim = 32
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size,
                    embedding_dim,
                    input_length = maxlen))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences = False)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [9]:
early_stop = EarlyStopping(monitor = 'val_loss', patience = 2)
history = model.fit(training_sequences,
                    y_train,
                    epochs = 30,
                    validation_data = (testing_sequences, y_test),
                    callbacks = [early_stop],
                    verbose = 2)

Epoch 1/30
1/1 - 6s - loss: 0.6932 - accuracy: 0.4545 - val_loss: 0.6908 - val_accuracy: 1.0000 - 6s/epoch - 6s/step
Epoch 2/30
1/1 - 0s - loss: 0.6899 - accuracy: 0.7273 - val_loss: 0.6931 - val_accuracy: 0.3333 - 246ms/epoch - 246ms/step
Epoch 3/30
1/1 - 0s - loss: 0.6875 - accuracy: 0.5455 - val_loss: 0.6954 - val_accuracy: 0.3333 - 256ms/epoch - 256ms/step


# New Section

In [11]:
test_news = ["Alert! Alert! The ski is definitely going to fall by the end of this year!!!"]
test_news_seq = tokenizer.texts_to_sequences(test_news)
test_news_seq = pad_sequences( test_news_seq,
                              maxlen = maxlen,
                               padding = padding,
                               truncating = truncating)
print(test_news_seq.shape)
test_news_pred = model.predict(test_news_seq)
print(test_news_pred)

(1, 128)
1/1 [==============================] - 0s 46ms/step
[[0.5069227]]
